In [1]:
import requests
import json
import time
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()
api_key = os.getenv('THE_GRAPH_API_KEY')

url = f"https://gateway.thegraph.com/api/{api_key}/subgraphs/id/7nuSuPhUgKSg5uKRh8g5jyjZWE8DnvQw1mQtvDWpxrnh"

def retrieve_proposals():
    body = f"""
    {{
        proposals(
            first: 1,
            orderBy: creationTime,
            orderDirection: desc
        ) {{
            id
            creationTime
            abstainDelegateVotes
            abstainWeightedVotes
            againstDelegateVotes
            againstWeightedVotes
            executionTime
            description
            forDelegateVotes
            forWeightedVotes
            proposer {{
              id
            }}
            quorumVotes
            state
            totalDelegateVotes
            totalWeightedVotes
        }}
    }}
    """
    
    response = requests.post(url=url, json={"query": body})
    
    if response.status_code == 200:
        return response.json()
    else:
        return []

In [ ]:
with open('../data/proposals/compound/compound_proposals.json', 'w', encoding="utf8") as outfile:
    json.dump(retrieve_proposals(), outfile, ensure_ascii=False)

In [6]:
with open('../data/proposals/compound/compound_proposals_1_year.json', 'r', encoding="utf8") as file:
    data = json.load(file)

In [9]:
url = f"https://gateway.thegraph.com/api/{api_key}/subgraphs/id/7nuSuPhUgKSg5uKRh8g5jyjZWE8DnvQw1mQtvDWpxrnh"

def retrieve_votes(proposal):
    body = f"""
    {{
        votes(
            where: {{
                proposal: "{proposal}",
            }},
            first: 1000,
            orderBy: weight,
            orderDirection: desc
        ) {{
            voter{{
                id
            }}
            choice
            weight
        }}
    }}
    """
    
    response = requests.post(url=url, json={"query": body})
    
    if response.status_code == 200:
        return response.json().get("data", {}).get("votes", [])
    else:
        return []

In [ ]:
all_votes = []

for proposal in data['data']['proposals']:
    proposal_id = proposal['id']
    votes = retrieve_votes(proposal_id)
    
    all_votes.append({
        "proposal_id": proposal_id,
        "votes": votes
    })
    
with open('../data/proposals/compound/compound_votes_1_year.json', 'w', encoding='utf8') as outfile:
    json.dump(all_votes, outfile, ensure_ascii=False)